# Outlier detection

In [1]:
# https://github.com/fnielsen/everything
from everything import *

In [2]:
# Read dataframe with features for companies
df = read_csv(expanduser('~/features.csv'), encoding='utf-8', index_col=0)

In [3]:
# Feature names
df.columns

Index([u'antal_ansatte', u'antal_penheder', u'reklamebeskyttet',
       u'sidste_virksomhedsstatus', u'virksomhedsform'],
      dtype='object')

In [4]:
# Functions for conversion to numerical dataframes
def to_dummies(df, column):
    datatype = df[column].dtypes
    if datatype in [int64, float64]:
        return df[[column]]
    elif datatype == bool:
        return df[[column]].astype(int)
    elif datatype == 'object':
        df_column = df[column].str.get_dummies()
        df_column.columns = [column + ":" + col for col in df_column.columns]
        return df_column
    else:
        raise ValueError('Unrecognized datatype for column {}'.format(column))
        
def dataframe_to_numerical(df):
    df_numerical = DataFrame(index=df.index)
    for column in df.columns:
        df_numerical = df_numerical.join(to_dummies(df, column))
    return df_numerical

In [5]:
# Numerical dataframe
dfn = dataframe_to_numerical(df)

In [6]:
# Preprocessing
imputer = Imputer()
scaler = StandardScaler(with_mean=False)
dfni = scaler.fit_transform(imputer.fit_transform(dfn))

In [8]:
# Outlier detection
one_class_svm = OneClassSVM()
one_class_svm.fit(dfni)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [9]:
decisions = one_class_svm.decision_function(dfni)

In [10]:
indices = argsort(decisions, axis=0)[:, 0]

In [46]:
dfn.iloc[indices[:30], :]

,antal_ansatte,antal_penheder,reklamebeskyttet,sidste_virksomhedsstatus:NORMAL,sidste_virksomhedsstatus:OPLØST EFTER ERKLÆRING,sidste_virksomhedsstatus:OPLØST EFTER FRIVILLIG LIKVIDATION,sidste_virksomhedsstatus:OPLØST EFTER FUSION,sidste_virksomhedsstatus:OPLØST EFTER KONKURS,sidste_virksomhedsstatus:OPLØST EFTER SPALTNING,sidste_virksomhedsstatus:SLETTET,...,virksomhedsform:Iværksætterselskab,virksomhedsform:Kommanditaktieselskab/Partnerselskab,virksomhedsform:Kommanditselskab,virksomhedsform:Partrederi,virksomhedsform:Personligt ejet Mindre Virksomhed,virksomhedsform:Primærkommune,virksomhedsform:Selskab med begrænset ansvar,virksomhedsform:Statslig administrativ enhed,virksomhedsform:Uoplyst virksomhedsform,virksomhedsform:Øvrige virksomhedsformer
cvr_nummer,,,,,,,,,,,,,,,,,,,,,
17146815,100,71,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
25042557,NaN,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60756112,500,5,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30580834,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29776946,200,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
26439272,200,4,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
65409313,1000,31,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
70042517,NaN,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
33341016,0,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [70]:
clusterer = MiniBatchKMeans(n_clusters=20)
clusterer.fit(dfni)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [71]:
distances = sum((dfni - clusterer.cluster_centers_[clusterer.labels_, :]) ** 2, axis=1)
indices_clusterer = argsort(-distances)

In [75]:
dfn.iloc[indices_clusterer[:20], :]

,antal_ansatte,antal_penheder,reklamebeskyttet,sidste_virksomhedsstatus:NORMAL,sidste_virksomhedsstatus:OPLØST EFTER ERKLÆRING,sidste_virksomhedsstatus:OPLØST EFTER FRIVILLIG LIKVIDATION,sidste_virksomhedsstatus:OPLØST EFTER FUSION,sidste_virksomhedsstatus:OPLØST EFTER KONKURS,sidste_virksomhedsstatus:OPLØST EFTER SPALTNING,sidste_virksomhedsstatus:SLETTET,...,virksomhedsform:Iværksætterselskab,virksomhedsform:Kommanditaktieselskab/Partnerselskab,virksomhedsform:Kommanditselskab,virksomhedsform:Partrederi,virksomhedsform:Personligt ejet Mindre Virksomhed,virksomhedsform:Primærkommune,virksomhedsform:Selskab med begrænset ansvar,virksomhedsform:Statslig administrativ enhed,virksomhedsform:Uoplyst virksomhedsform,virksomhedsform:Øvrige virksomhedsformer
cvr_nummer,,,,,,,,,,,,,,,,,,,,,
25042557,NaN,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32364152,NaN,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31851734,NaN,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17146815,100,71,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29776946,200,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
87217418,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89970113,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15964049,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30580834,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [76]:
# Investigate cluster model as a function of number of clusters
inertias = []
for n_clusters in range(1, 50):
    clusterer = MiniBatchKMeans(n_clusters=n_clusters, max_iter=200, max_no_improvement=30, n_init=10)
    clusterer.fit(dfni)
    inertias.append(clusterer.inertia_)

In [77]:
plot(inertias)
show()